# Characters — Notes Companion

**6.7970/8.750 Symmetry and its Application to Machine Learning**

This notebook accompanies the [Characters notes](https://symm4ml.mit.edu/notes/characters). Run these examples to build character tables and decompose reducible representations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/characters_notes_companion.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [ ]:
import numpy as np
from symm4ml import groups, linalg, rep
from symm4ml.utils import match_character_tables, print_character_table

## Reference: $P(3)$ data

In [ ]:
# Build P(3) table, find irreps, and get conjugacy classes
p3_matrices = groups.permutation_matrices(3)
table = groups.make_multiplication_table(p3_matrices)
irreps = rep.infer_irreps(table)
conj_classes = groups.conjugacy_classes(table)

print(f"P(3): h = {len(table)}, {len(irreps)} irreps, {len(conj_classes)} conjugacy classes")
print(f"Irrep dimensions: {[ir.shape[1] for ir in irreps]}")
print(f"Conjugacy classes: {conj_classes}")

---
## Example: Building the Character Table

The **character** of a representation is the trace: $\chi(g) = \text{tr}(D(g))$.
Since the trace is invariant under similarity transforms, the character captures the essential information of a representation.

The function `rep.character_table(irreps, conj_classes)` computes the full character table.

In [ ]:
# Build the character table
char_table = rep.character_table(irreps, conj_classes)

# Reference character table for P(3) ≅ S₃ (Dresselhaus convention)
# Columns: E, 2C₃ (rotations), 3σ (mirrors)
names = ['A₁', 'A₂', 'E']
class_names = ['E', '2C₃', '3σ']
ref_chars = np.array([
    [1,  1,  1],   # A₁ (trivial)
    [1,  1, -1],   # A₂ (sign)
    [2, -1,  0],   # E  (standard)
])

# Align computed table to reference convention
row_perm, col_perm = match_character_tables(char_table, ref_chars)
irreps = [irreps[i] for i in row_perm]
conj_classes_list = list(conj_classes)
conj_classes_list = [conj_classes_list[j] for j in col_perm]
char_table = rep.character_table(irreps, conj_classes_list)

print_character_table(char_table, names, class_names,
                      title="Character table of P(3) ≅ S₃")

In [ ]:
# The first column (identity class) gives the dimension of each irrep:
# χ(E) = tr(I_{ℓ_j}) = ℓ_j
print("χ(E) for each irrep (= dimension):")
for i, name in enumerate(names):
    print(f"  {name}: χ(E) = {char_table[i, 0].real:.0f}  (dim = {irreps[i].shape[1]})")

In [ ]:
# Verify the Wonderful Orthogonality Theorem holds
print(f"WOT satisfied? {rep.check_orthogonality_theorem(irreps)}")

---
## Decomposing with the Character Table

The decomposition formula tells us how many times each irrep $\Gamma_j$ appears in a reducible representation:

$$a_j = \frac{1}{h} \sum_k N_k \left[\chi^{(\Gamma_j)}(C_k)\right]^* \chi(C_k)$$

We only need the **character table** and the **characters of the reducible representation** — no matrix diagonalization required!

In [ ]:
def decompose_by_characters(char_table, class_sizes, chi_reducible):
    """Decompose a reducible representation using the character orthogonality formula.

    Args:
        char_table: Character table, shape [num_irreps, num_classes].
                    Rows = irreps, columns = conjugacy classes.
        class_sizes: List/array of N_k (number of elements in each class).
        chi_reducible: Characters of the reducible representation for each class,
                       shape [num_classes].

    Returns:
        Array of multiplicities a_j (how many times each irrep appears).
    """
    h = sum(class_sizes)  # group order
    class_sizes = np.array(class_sizes)
    chi_reducible = np.array(chi_reducible)

    # a_j = (1/h) * sum_k  N_k * conj(chi^(Gamma_j)(C_k)) * chi(C_k)
    a = np.array([
        np.sum(class_sizes * np.conj(char_table[j]) * chi_reducible) / h
        for j in range(char_table.shape[0])
    ])
    return np.round(a.real).astype(int)

In [ ]:
# Class sizes N_k (matching the column order of our character table: E, 2C₃, 3σ)
class_sizes = [len(c) for c in conj_classes_list]
print(f"Class sizes N_k: {class_sizes}")
print(f"Group order h = {sum(class_sizes)}")
print()

# Characters of the 3D permutation representation for each class
# χ(E) = 3, χ(C₃) = 0, χ(σ) = 1
perm_rep = groups.permutation_matrices(3)
chi_perm = np.array([np.trace(perm_rep[next(iter(c))]).real for c in conj_classes_list])
print(f"χ(3D perm) per class: {chi_perm}")
print()

# Decompose using the character formula
multiplicities = decompose_by_characters(char_table, class_sizes, chi_perm)

print("Decomposition formula a_j = (1/h) Σ_k N_k χ*(Γ_j) χ(C_k):")
for j, name in enumerate(names):
    terms = [f"({class_sizes[k]})({char_table[j,k].real: .0f})({chi_perm[k]:.0f})"
             for k in range(len(class_sizes))]
    s_j = sum(class_sizes[k] * char_table[j,k].real * chi_perm[k] for k in range(len(class_sizes)))
    print(f"  a_{name} = (1/6)[{' + '.join(terms)}] = {s_j:.0f}/6 = {multiplicities[j]}")

print(f"\n3D perm rep = {' ⊕ '.join(name for j, name in enumerate(names) if multiplicities[j] > 0)}")

---
## Verification: Matrix-Based Decomposition

We can verify the character-based result by directly decomposing the representation matrices using `rep.decompose_rep_into_irreps`.

In [ ]:
# Build the 3D permutation representation
perm_rep = groups.permutation_matrices(3)  # shape [6, 3, 3]

# Is it irreducible?
print(f"Permutation rep dimension: {perm_rep.shape[1]}")
print(f"Is it an irrep? {rep.is_an_irrep(table, perm_rep)}")

In [ ]:
# Decompose into irreps
sub_irreps = rep.decompose_rep_into_irreps(perm_rep)

print(f"Decomposes into {len(sub_irreps)} irreps:")
for i, sub in enumerate(sub_irreps):
    print(f"  Sub-irrep {i+1}: dimension {sub.shape[1]}")

In [ ]:
# Identify which known irreps they correspond to
print("Identifying sub-irreps:")
for i, sub in enumerate(sub_irreps):
    for j, known in enumerate(irreps):
        if rep.are_isomorphic(sub, known):
            print(f"  Sub-irrep {i+1} ≅ {names[j]} (dim {known.shape[1]})")
            break

print("\nThis matches the hand calculation: perm rep = A₁ ⊕ E")